<a href="https://colab.research.google.com/github/7717CMI/Automate/blob/main/ExcelAutomate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gspread oauth2client pandas


In [13]:
import pandas as pd

# Define column names
columns = ['RID', 'Domain', 'Sub Domain', 'Keywords', 'AVG. Search', 'Competition']

# Create a list of data
data = [
    [1560, 'ICT', 'Industrial Automation and Machinery', 'Weather Radar Market', 5000000, 'Low'],
    [7675, 'CMFE', 'Consumer Goods', 'Museum Market', 5000000, 'Low'],
    [913, 'CMFE', 'Consumer Goods', 'Spa Market', 5000000, 'Low'],
    [5676, 'ICT', 'Semiconductors', 'Uninterruptible Power Supply Market', 5000000, 'Low'],
    [66,'HC','Medical Devices', 'Tonometry Device Market','500','High'],
]

# Create a pandas DataFrame with the given values
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df)


    RID Domain                           Sub Domain  \
0  1560    ICT  Industrial Automation and Machinery   
1  7675   CMFE                       Consumer Goods   
2   913   CMFE                       Consumer Goods   
3  5676    ICT                       Semiconductors   
4    66     HC                      Medical Devices   

                              Keywords AVG. Search Competition  
0                 Weather Radar Market     5000000         Low  
1                        Museum Market     5000000         Low  
2                           Spa Market     5000000         Low  
3  Uninterruptible Power Supply Market     5000000         Low  
4              Tonometry Device Market         500        High  


In [14]:

!pip install pandas
import pandas as pd
from google.colab import data_table


# Enable the interactive display
data_table.enable_dataframe_formatter()

# Display the DataFrame using data_table
df

,RID,Domain,Sub Domain,Keywords,AVG. Search,Competition
0,1560,ICT,Industrial Automation and Machinery,Weather Radar Market,5000000,Low
1,7675,CMFE,Consumer Goods,Museum Market,5000000,Low
2,913,CMFE,Consumer Goods,Spa Market,5000000,Low
3,5676,ICT,Semiconductors,Uninterruptible Power Supply Market,5000000,Low
4,66,HC,Medical Devices,Tonometry Device Market,500,High


In [16]:
df['AVG. Search'] = pd.to_numeric(df['AVG. Search'], errors='coerce')

In [17]:
filtered_df = df[(df['AVG. Search'] >= 5000) & (df['Competition'] == 'Low')]

# Select only the 'RID' and 'Keywords' columns
output_df = filtered_df[['RID', 'Keywords']]

# Save the filtered data to a new Excel file named "Output.xlsx"
output_df.to_excel('Output.xlsx', index=False)

print("Filtered data has been saved to 'Output.xlsx'")

Filtered data has been saved to 'Output.xlsx'


In [21]:
import pandas as pd
import time
import requests
import json

# Define the GoogleTrendsAgent class (same as in the previous code you provided)

class GoogleTrendsAgent:
    def __init__(self, serp_api_key):
        self.api_key = serp_api_key
        self.base_url = "https://serpapi.com/search.json"

    def get_trends_data(self, keyword):
        """Get interest over time for a keyword"""
        params = {
            'engine': 'google_trends',
            'q': keyword,
            'data_type': 'TIMESERIES',
            'date': 'now 7-d',  # Last 7 days
            'geo': '',  # Worldwide
            'api_key': self.api_key
        }

        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            data = response.json()

            # Check for API errors
            if 'error' in data:
                print(f"API Error for {keyword}: {data['error']}")
                return []

            # Correct path based on API documentation
            return data.get('interest_over_time', {}).get('timeline_data', [])
        except Exception as e:
            print(f"Error fetching trends for {keyword}: {e}")
            return []

    def get_region_data(self, keyword):
        """Get interest by region for a keyword"""
        params = {
            'engine': 'google_trends',
            'q': keyword,
            'data_type': 'GEO_MAP_0',  # Correct data_type for interest by region
            'date': 'now 7-d',
            'api_key': self.api_key
        }

        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            data = response.json()

            # Check for API errors
            if 'error' in data:
                print(f"API Error for {keyword} regions: {data['error']}")
                return []

            # Correct path based on API documentation
            return data.get('interest_by_region', [])
        except Exception as e:
            print(f"Error fetching regions for {keyword}: {e}")
            return []

    def check_threshold_criteria(self, interest_values):
        """Check if interest > 50 appears at least twice in 7 days"""
        if not interest_values:
            return False
        days_above_50 = sum(1 for value in interest_values if value and value > 50)
        return days_above_50 >= 2

    def check_country_criteria(self, region_data):
        """Check if USA, Germany, Japan, UK, or France are in top countries"""
        if not region_data:
            return False

        target_countries = ['United States', 'Germany', 'Japan', 'United Kingdom', 'France',"China"]
        top_countries = [item.get('location', '') for item in region_data[:10]]
        return any(country in top_countries for country in target_countries)

    def extract_interest_values(self, trends_data, keyword):
        """Extract interest values from trends data for a single keyword"""
        interest_values = []

        for item in trends_data:
            values = item.get('values', [])
            if values:
                # For single keyword queries, there should be one value per time period
                if len(values) == 1:
                    interest_values.append(values[0].get('extracted_value', 0))
                else:
                    # If multiple values, find the one for our keyword
                    for value in values:
                        if value.get('query', '').lower() == keyword.lower():
                            interest_values.append(value.get('extracted_value', 0))
                            break
                    else:
                        # If keyword not found, append 0
                        interest_values.append(0)

        return interest_values

    def analyze_keyword(self, keyword):
        """Analyze a single keyword"""
        print(f"Analyzing: {keyword}")

        # Get trends data
        trends_data = self.get_trends_data(keyword)
        if not trends_data:
            print(f"No trends data for {keyword}")
            return None

        # Extract interest values
        interest_values = self.extract_interest_values(trends_data, keyword)

        if len(interest_values) < 7:
            print(f"Not enough data for {keyword} (got {len(interest_values)} data points)")
            return None

        # Get region data with rate limiting
        time.sleep(1)  # Rate limiting between API calls
        region_data = self.get_region_data(keyword)

        # Apply your logic
        meets_threshold = self.check_threshold_criteria(interest_values)
        has_target_countries = self.check_country_criteria(region_data)

        print(f"  Threshold check (>50 twice): {meets_threshold}")
        print(f"  Target countries check: {has_target_countries}")
        print(f"  Interest pattern: {' → '.join(map(str, interest_values))}")

        if meets_threshold and has_target_countries:
            return {
                'keyword': keyword,
                'interest_values': interest_values,
                'max_interest': max(interest_values) if interest_values else 0,
                'avg_interest': round(sum(interest_values) / len(interest_values)) if interest_values else 0,
                'days_above_50': sum(1 for v in interest_values if v and v > 50),
                'top_countries': [item.get('location', '') for item in region_data[:5]]
            }

        return None

    def process_keywords(self, keywords):
        """Process list of keywords and return only those meeting criteria"""
        qualifying_keywords = []

        for i, keyword in enumerate(keywords):
            keyword = keyword.strip()
            if not keyword:
                continue

            print(f"\n[{i+1}/{len(keywords)}] Processing: {keyword}")

            result = self.analyze_keyword(keyword)
            if result:
                qualifying_keywords.append(result)
                print(f"✅ {keyword} qualifies!")
            else:
                print(f"❌ {keyword} doesn't meet criteria")

            # Rate limiting between keywords
            if i < len(keywords) - 1:  # Don't sleep after the last keyword
                time.sleep(2)

        return qualifying_keywords

# Read the "Output.xlsx" to extract keywords
df = pd.read_excel('Output.xlsx')  # Assuming the file "Output.xlsx" was generated in your earlier code

# Extract the "Keywords" column
keywords = df['Keywords'].tolist()

# Print the extracted keywords
print(f"Extracted Keywords: {keywords}")

# Now initialize the GoogleTrendsAgent and process the keywords
API_KEY = "a7bfa36977090c156c1becb783a42c6fff3d443a94330794a191c19303f84a48"  # Replace with your actual API key
agent = GoogleTrendsAgent(API_KEY)

# Process the extracted keywords
results = agent.process_keywords(keywords)

# Display final results
print(f"\n🎯 FINAL RESULTS: {len(results)} out of {len(keywords)} keywords qualify")
if results:
    for i, result in enumerate(results, 1):
        print(f"\n📈 [{i}] {result['keyword'].upper()}")
        print(f"   📊 Max Interest: {result['max_interest']}%")
        print(f"   📊 Avg Interest: {result['avg_interest']}%")
        print(f"   🔥 Days >50: {result['days_above_50']}")
        print(f"   📅 7-day pattern: {' → '.join(map(str, result['interest_values']))}")
        print(f"   🌍 Top countries: {', '.join(result['top_countries'])}")
else:
    print("❌ No keywords met the criteria.")
    print("\n💡 Tips:")
    print("   • Try more popular/trending keywords")
    print("   • Check if keywords have sufficient search volume")
    print("   • Verify your API key has sufficient credits")


Extracted Keywords: ['Weather Radar Market', 'Museum Market', 'Spa Market', 'Uninterruptible Power Supply Market']

[1/4] Processing: Weather Radar Market
Analyzing: Weather Radar Market
  Threshold check (>50 twice): True
  Target countries check: True
  Interest pattern: 0 → 0 → 0 → 0 → 0 → 0 → 0 → 68 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 98 → 0 → 0 → 0 → 0 → 0 → 76 → 0 → 0 → 0 → 0 → 0 → 0 → 100 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 65 → 0 → 0 → 0 → 0 → 0 → 55 → 0 → 77 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 76 → 59 → 0 → 0 → 75 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0 → 0
✅ Weather Radar Market qualifies!

[2/4] 